# Implementation of WGN+SW for simulation with adversarial perturbations (Figure 7)

This notebook provides the implementation of Wang et al. 2020 paper for our settings and datasets - most of the code is the same as the official implementation - https://github.com/wenshuoguo/robust-fairness-code

In [ ]:
%load_ext autoreload
%autoreload 2

# general imports
import numpy as np
import sys, random
import os, warnings
import itertools as iter
from copy import deepcopy
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
from sklearn.linear_model import LogisticRegression
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=FutureWarning)
np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)

# initialization
rng = np.random.default_rng(8735) #(1234)
random.seed()

import itertools
import pickle

# add to path
print(os.getcwd())
sys.path.append(os.getcwd())

# import ai360
from aif360.datasets import BinaryLabelDataset
from aif360.datasets import AdultDataset, CompasDataset
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_adult, load_preproc_data_compas

# import main code
from utils import *
import algorithms as denoisedfair
from lamy_noise_fairlearn.util import *
from awasthi_equalized_odds_under_perturbation.equalized_odds import *

from scipy.optimize import minimize
from joblib import Parallel, delayed

sys.path.append(os.getcwd()+'/robust-fairness-code/')
from softweights_training import *
import dro_training, naive_training, data, losses, optimization, model, rbc_utils

In [ ]:
exec(open('baseSimulation.py').read())

In [ ]:
result_folder = 'results'
os.system(f'mkdir {result_folder}')

## Code for simulation

In [ ]:
verbose = False

In [ ]:
def test_sw_flipping_parallel(eta0=0.3, eta1=0.1, flip_func = flipping, d='compas', reps=100):
    results_dicts_runs = []  
    
    constraint = 'fpr'
    learning_rates_theta = [0.001, 0.01, 0.1]
    learning_rates_lambda = [0.5, 1.0, 2.0]
    learning_rates_W =   [0.01, 0.1]
    num_runs=1
    minibatch_size=None
    num_iterations_per_loop= 25
    num_loops= 30
    constraints_slack=0.0
    num_avg_iters=0
    optimize_robust_constraints=False
    rank_objectives=False # parameters for find_best_candidate_index
    max_constraints=False # parameters for find_best_candidate_index
    num_iterations_W=5
    max_diff=0.05 #0.01
    best_index_nburn=0 # Number of initial candidate indices to exclude from find_best_candidate_index.
    seed_start=100

    def select_job(job_id, ss):
        np.random.seed(job_id+1234)
        
        if ss==None: rng_loc = rng
        else: rng_loc = np.random.default_rng(ss)
        
        attr = "sex"
        if d == "compas": dataset = load_preproc_data_compas()
        else: raise NotImplementedError

        protected_name = attr

        dataset_train, dataset_test = dataset.split([0.7], shuffle=True)
        train_labels = [1-int(lab[0]) for lab in dataset_train.labels]
        test_labels = [1-int(lab[0]) for lab in dataset_test.labels]

        index, noisyfea, eta_group = flip_func(dataset_train.feature_names, dataset_train.features, train_labels, protected_name, eta0, eta1, rng)
        index, test_noisyfea, _ = flip_func(dataset_test.feature_names, dataset_test.features, test_labels, protected_name, eta0, eta1, rng)
        
        print(f'ETAgroup {eta_group}')

        feature_names = dataset_train.feature_names
        del(feature_names[index])
        feature_names = feature_names + ["attr_1", "attr_2"] + ["proxy_1", "proxy_2"]

        train_groups = list(dataset_train.features[:,index])
        test_groups = list(dataset_test.features[:,index])

        dataset_train = dataset_train.features 
        dataset_train = np.delete(dataset_train, index, 1)
        dataset_test = dataset_test.features
        dataset_test = np.delete(dataset_test, index, 1)

        feat_n = [[1] if train_groups[i] == 1 else [0] for i in range(len(train_groups))]
        feat_a = [[1] if train_groups[i] == 0 else [0] for i in range(len(train_groups))]

        dataset_train = np.hstack((dataset_train, feat_n))
        dataset_train = np.hstack((dataset_train, feat_a))

        dataset_noisy = np.copy(dataset_train)

        feat_n = [[1] if noisyfea[i] == 1 else [0] for i in range(len(noisyfea))]
        feat_a = [[1] if noisyfea[i] == 0 else [0] for i in range(len(noisyfea))]
        dataset_noisy = np.hstack((dataset_noisy, feat_n))
        dataset_noisy = np.hstack((dataset_noisy, feat_a))

        feat_n = [[1] if test_groups[i] == 1 else [0] for i in range(len(test_groups))]
        feat_a = [[1] if test_groups[i] == 0 else [0] for i in range(len(test_groups))]

        dataset_test = np.hstack((dataset_test, feat_n))
        dataset_test = np.hstack((dataset_test, feat_a))

        dataset_noisy_test = np.copy(dataset_test)
        feat_n = [[1] if test_noisyfea[i] == 1 else [0] for i in range(len(test_noisyfea))]
        feat_a = [[1] if test_noisyfea[i] == 0 else [0] for i in range(len(test_noisyfea))]
        dataset_noisy_test = np.hstack((dataset_noisy_test, feat_n))
        dataset_noisy_test = np.hstack((dataset_noisy_test, feat_a))

        dataset_train_pd = pd.DataFrame(dataset_noisy, columns=feature_names)
        dataset_train_pd['label'] = train_labels
        train_df = dataset_train_pd.copy()

        dataset_test_pd = pd.DataFrame(dataset_noisy_test, columns=feature_names)
        dataset_test_pd['label'] = test_labels
        test_df = dataset_test_pd.copy()

        val_df = train_df.copy()

        label_column = "label" 
        feature_names = list(dataset_train_pd.columns)
        feature_names.remove(label_column)
        protected_columns = ["attr_1", "attr_2"]

        proxy_columns = ["proxy_1", "proxy_2"]
        b = build_b(train_df, proxy_columns, protected_columns)
        true_group_marginals = get_true_group_marginals(train_df, protected_columns)

        val_objectives = []
        val_constraints_matrix = []
        results_dicts = []
        learning_rates_iters_theta = []
        learning_rates_iters_lambda = []
        learning_rates_iters_W = []

        results = []
        results_train = []
        result_details = []
        
        bar = tqdm(itertools.product(learning_rates_theta, learning_rates_lambda, learning_rates_W))
        for learning_rate_theta, learning_rate_lambda, learning_rate_W in bar:
            bar.set_description("Starting optimizing LR theta: %.3f, LR lambda: %.3f, LR W: %.3f" % (learning_rate_theta, learning_rate_lambda, learning_rate_W))
            bar.refresh() # to show immediately the update
            sw_model = SoftweightsHeuristicModel(b, true_group_marginals, feature_names, proxy_columns, label_column, maximum_lambda_radius=1.0)
            sw_model.build_train_ops(constraint=constraint, learning_rate_theta=learning_rate_theta, learning_rate_lambda=learning_rate_lambda, learning_rate_W=learning_rate_W, constraints_slack=constraints_slack)

            # training_helper returns the list of errors and violations over each epoch.
            results_dict = training_helper(
                  sw_model,
                  train_df,
                  val_df,
                  test_df,
                  protected_columns, 
                  proxy_columns, 
                  label_column,
                  minibatch_size=minibatch_size,
                  num_iterations_per_loop=num_iterations_per_loop,
                  num_loops=num_loops,
                  optimize_robust_constraints=optimize_robust_constraints,
                  num_iterations_W=num_iterations_W,
                  max_diff=max_diff,
                  constraint=constraint)

            
            # Get best iterate using training set.
            best_index_iters = rbc_utils.find_best_candidate_index(np.array(results_dict['train_01_objective_vector'][best_index_nburn:]),np.array(results_dict['train_01_robust_constraints_matrix'][best_index_nburn:]), rank_objectives=rank_objectives, max_constraints=max_constraints)
            best_index_iters = best_index_iters + best_index_nburn
            results_dict_best_idx = add_results_dict_best_idx(results_dict, best_index_iters)
            results_dicts.append(results_dict_best_idx)
            if num_avg_iters == 0:
                best_val_objective = results_dict['val_01_objective_vector'][best_index_iters]
                best_val_constraints = results_dict['val_01_true_G_constraints_matrix'][best_index_iters]
                val_objectives.append(best_val_objective)
                val_constraints_matrix.append(best_val_constraints)
                
                stats = results_dict['stat_test_vector'][best_index_iters]
                stats_train = results_dict['stat_train_vector'][best_index_iters]
                results.append(stats)
                results_train.append(stats_train)
                print(f'outside stats: {results_train}')
            else: 
                assert(num_avg_iters > 0)
                avg_val_objective = np.mean(np.array(results_dict['val_01_objective_vector'][-num_avg_iters:]))
                val_objectives.append(avg_val_objective)
                avg_val_constraints = np.mean(np.array(results_dict['val_01_robust_constraints_matrix'][-num_avg_iters:]), axis=0)
                val_constraints_matrix.append(avg_val_constraints)
            
            learning_rates_iters_theta.append(learning_rate_theta)
            learning_rates_iters_lambda.append(learning_rate_lambda)
            learning_rates_iters_W.append(learning_rate_W)
            
            result_details.append(results_dict)

        best_index = rbc_utils.find_best_candidate_index(np.array(val_objectives),\
                                                         np.array(val_constraints_matrix),\
                                                         rank_objectives=rank_objectives,\
                                                         max_constraints=max_constraints)
        best_stats = results[best_index]
        best_stats_train = results_train[best_index]
        best_learning_rate_theta = learning_rates_iters_theta[best_index]
        best_learning_rate_lambda = learning_rates_iters_lambda[best_index]
        best_learning_rate_W = learning_rates_iters_W[best_index]
            
        print(f'Result one rep: {best_stats} @ θ:{best_learning_rate_theta} λ:{best_learning_rate_lambda} W:{best_learning_rate_W}')
        print(f'Result one rep train: {best_stats_train}')
        
        return results, results_train, result_details, best_stats
        
    # reps = 100
    all_results = []
    all_results_train = []
    all_result_details = []
    all_best_stats = []
    
    CORES = 10
    
    ss = rng.bit_generator._seed_seq ## seed sequence (source: https://albertcthomas.github.io/good-practices-random-number-g
    child_states = ss.spawn(reps) ## child sequences
    
    answer = Parallel(n_jobs=CORES, verbose=100)(delayed(select_job)(int(i), child_states[i]) for i in range(reps))
    
    
    
    for i in range(reps):
        all_results.append(answer[i][0])
        all_results_train.append(answer[i][1])
        all_result_details.append(answer[i][2])
        all_best_stats.append(answer[i][3])
    
    
    acc = []; sr = []; fpr = []; fdr = []; tpr = [];
    for t in all_best_stats: 
        if t['sr']==0: continue
        acc.append(t['acc']); sr.append(t['sr']); fpr.append(t['fpr']); 
        fdr.append(t['fdr']); tpr.append(t['tpr']); 

    print('')
    print('')
    
    print(all_best_stats)
    
    print('')
    print('')
    
    print(f"Results: acc={np.mean(acc)} std={np.std(acc)}")
    print(f"\tSR={np.mean(sr)} std={np.std(sr)}")
    print(f"\tFPR={np.mean(fpr)} std={np.std(fpr)}")
    print(f"\tFDR={np.mean(fdr)} std={np.std(fdr)}")
    print(f"\tTPR={np.mean(tpr)} std={np.std(tpr)}")
    
    
    f = open(f'{result_folder}/WGN+SW-internal', 'wb')
    pickle.dump(all_results, f)
    f = open(f'{result_folder}/WGN+SW-internal-train', 'wb')
    pickle.dump(all_results_train, f)
    f = open(f'{result_folder}/WGN+SW-internal-details', 'wb')
    pickle.dump(all_result_details, f)
    
    return all_results, all_results_train, all_result_details, all_best_stats

## Runnning simulation

In [ ]:
reps = 100

### Adversary: A_TN

In [ ]:
st = time.time()
all_results, all_results_train, all_result_details, all_best_stats = test_sw_flipping_parallel(eta0=0.0175,\
                                                                                               eta1=0.0,\
                                                                                               flip_func=flipping_far_from_boundary_TN,\
                                                                                               reps=reps)
print(f'Time taken: {time.time() - st}')

f = open(f'{result_folder}/WGN+SW-TN-{reps}iters', 'wb')
pickle.dump(all_results, f)
f = open(f'{result_folder}/WGN+SW-TN-{reps}iters-train', 'wb')
pickle.dump(all_results_train, f)
f = open(f'{result_folder}/WGN+SW-TN-{reps}iters-details', 'wb')
pickle.dump(all_result_details, f)
f = open(f'{result_folder}/WGN+SW-TN-{reps}iters-best', 'wb')
pickle.dump(all_best_stats, f)

### Adversary: A_FN

In [ ]:
st = time.time()
all_results, all_results_train, all_result_details, all_best_stats = test_sw_flipping_parallel(eta0=0.0175, \
                                                                                               eta1=0.0,\
                                                                                               flip_func=flipping_far_from_boundary_FN,\
                                                                                               reps=reps)
print(f'Time taken: {time.time() - st}')

f = open(f'{result_folder}/WGN+SW-FN-{reps}iters', 'wb')
pickle.dump(all_results, f)
f = open(f'{result_folder}/WGN+SW-FN-{reps}iters-train', 'wb')
pickle.dump(all_results_train, f)
f = open(f'{result_folder}/WGN+SW-FN-{reps}iters-details', 'wb')
pickle.dump(all_result_details, f)
f = open(f'{result_folder}/WGN+SW-FN-{reps}iters-best', 'wb')
pickle.dump(all_best_stats, f)